In [25]:
#Business Search      URL -- 'https://api.yelp.com/v3/businesses/search'
#Business Match       URL -- 'https://api.yelp.com/v3/businesses/matches'
#Phone Search         URL -- 'https://api.yelp.com/v3/businesses/search/phone'

#Business Details     URL -- 'https://api.yelp.com/v3/businesses/{id}'
#Business Reviews     URL -- 'https://api.yelp.com/v3/businesses/{id}/reviews'
#Delivary                     https://api.yelp.com/v3/transactions/{transaction_type}/search
#Businesses, Total, Region

# Import the modules
import requests
import json
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import requests 
import time
import json
import seaborn 

# connect to API key
from config import yelpkey

# Define API Key, Endpoint, and Header
API_KEY = yelpkey
ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
HEADERS = {'Authorization': 'bearer %s' % API_KEY}

# Let's first do an easy call to see how we find the total # of restaurants in a zip code

# Define parameters of  search: chinese restaurants in zipcode 10007
PARAMETERS = {'location': '10918',
              'limit': 50,
              'categories': 'restaurant,all'}
                

# Make a request to the Yelp API
response = requests.get(url = ENDPOINT,
                        params = PARAMETERS,
                        headers = HEADERS)

# Convert the JSON String
business_data = response.json()

# print some basic data on the response
biz = []

count_in_zip = 0
for restaurant in business_data['businesses']:
    if restaurant['location']['zip_code'] == PARAMETERS['location']:
        count_in_zip += 1
        biz.append(f'{restaurant["alias"]} -- DELIVERS: {"delivery" in restaurant["transactions"]}')

print(f'{num_of_results} search results total near {PARAMETERS["location"]}')
print(f'There are {count_in_zip} "{PARAMETERS["categories"].replace("+", " ")}" businesses directly in {PARAMETERS["location"]}:\n')
print(*biz, sep='\n')

614 search results total near 10918
There are 11 "restaurant,all" businesses directly in 10918:

allans-falafel-chester -- DELIVERS: False
christophers-bistro-chester -- DELIVERS: False
rustic-wheelhouse-restaurant-chester-3 -- DELIVERS: False
thank-goodness-its-vegan-chester -- DELIVERS: False
dos-latinos-chester -- DELIVERS: False
the-castle-fun-center-chester -- DELIVERS: False
chester-diner-chester -- DELIVERS: False
magoya-japanese-restaurant-chester -- DELIVERS: False
clayton-delaneys-dining-saloon-chester -- DELIVERS: False
sugarloaf-taphouse-chester -- DELIVERS: False
chester-ice-cream-chester -- DELIVERS: False


In [4]:
# we can see in the response above that there are three keys, one of which is "total"

# Print the value in the "total" key to see how many Chinese restaurants are in that zip code
print(business_data['total'])

227


In [ ]:
# so now let's apply that approach with a list of zip codes instead of just one

In [10]:
# import NY zip codes list from CSV & convert into dataframe
# note, to test, I'm using an abbreciated version of our zip code list 

zipcodes_data = "clean_operational_ny_zips_SHORT.csv" 
zipcodes_df = pd.read_csv(zipcodes_data)
zipcodes_df

,Zip Code
0,10001
1,10002
2,10003
3,10004
4,10005
5,10006
6,10007


In [11]:
# Add column to hold number of Chinese restaurants
zipcodes_df["# Chinese Rest"] = ""
zipcodes_df


,Zip Code,# Chinese Rest
0,10001,
1,10002,
2,10003,
3,10004,
4,10005,
5,10006,
6,10007,


In [26]:
# create a loop that looks up the zip in each row and adds total # Chinese rest to the column

for index, row in zipcodes_df.iterrows():    
    zip = row['Zip Code']
    PARAMETERS = {'categories': 'restaurants, all',
                  'location': zip,
                  'limit': 50,
                  'offset': 0}
    try:
        print(f"Finding restaurant info for zip code: {zip}")
        count_in_zip = 0
        if business_data['total'] > 50:
            for i in range(0, int(business_data['total']), 50):
                PARAMETERS['offset'] = i
                business_data = requests.get(url=ENDPOINT, params=PARAMETERS, headers=HEADERS).json()
                for restaurant in business_data['businesses']:
                    if restaurant['location']['zip_code'] == str(PARAMETERS['location']):
                        count_in_zip += 1
                        zipcodes_df.loc[index, "# Chinese Rest"] = count_in_zip
        else:
            for restaurant in business_data['businesses']:
                if restaurant['location']['zip_code'] == str(PARAMETERS['location']):
                    count_in_zip += 1
                    zipcodes_df.loc[index, "# Chinese Rest"] = count_in_zip
    except json.decoder.JSONDecodeError:
        print("Missing field/result... skipping.")
        
    print("-------------------------------------------")

NameError: name 'zipcodes_df' is not defined

In [23]:
# print the data frame, which should now include the Chinese restaurant count by zip
zipcodes_df


,Zip Code,# Chinese Rest
0,10001,6
1,10002,9
2,10003,16
3,10004,4
4,10005,4
5,10006,7
6,10007,4
